In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.grid_search import GridSearchCV

df = pd.read_csv('university_train.csv')

df = df[['MAIN', 'NUMBRANCH', 'HIGHDEG', 'CONTROL', 'DEPENDENT', 'AGE_ENTRY', 'MARRIED', 'MD_FAMINC', 'percent_on_student_loan']]

#bin the percentage of student in 25% increments - based loosely on the the standard deviation
def bin(df):
    dependents = df['DEPENDENT']
    try: 
        if dependents <= 0.25:
            return 25
        elif dependents <= 0.5099999 and dependents >= 0.2599999:
            return 50
        elif dependents <= 0.75 and dependents >= 0.51:
            return 75
        else:
            return 100
    except:
        pass

df['DEPENDENT_BIN'] = df['DEPENDENT'].apply(bin)


#group ages by under 20, 20-30, 30-40, 40-50,, 50+

def group(ages):   
    try: 
        if ages >= 19.0 and ages < 20.0: 
            return int(10)
        elif ages <= 29.9999999 and ages >= 20.0:
            return int(20)
        elif ages <= 39.9999999 and ages >= 30.0:
            return int(30)
        elif ages <= 49.999999 and ages >= 40.0:
            return int(40)
        elif ages >= 50 and ages <= 60.0:
            return int(50)
        else:
            return 'wtf'    
    except:
        pass

df['AGE_GROUPED'] = df['AGE_ENTRY'].apply(group)

def lump(marraige):   
    try: 
        if marraige >= 0.0 and marraige <= 0.10: 
            return int(0)
        elif marraige >= 0.100001 and marraige <= 0.20:
            return int(1)
        elif marraige >= 0.200001 and marraige <= 0.30:
            return int(2)
        elif marraige >= 0.300001 and marraige <= 0.40:
            return int(3)
        elif marraige >= 0.400001 and marraige <= 0.50:
            return int(4)
        elif marraige >= 0.500001 and marraige <= 0.60:
            return int(5)
        elif marraige >= 0.600001 and marraige <= 0.70:
            return int(6)
        else:
            return 'wtf'
    except:
        pass   

df['MARRAIGE_LUMPED'] = df['MARRIED'].apply(lump)

df['INCOME_TRANSFORMED'] = df['MD_FAMINC'].round(2)

def yay(moola):   
    try: 
        if moola >= 0.00 and moola <= 17241.75: 
            return int(1)
        elif moola <= 34483.50 and moola >= 17241.76:
            return int(2)
        elif moola <= 51725.25 and moola >= 34483.51:
            return int(3)
        elif moola <= 68967.00 and moola >= 51725.26:
            return int(4)
        elif moola <= 86208.75 and moola >= 68967.01:
            return int(5)
        elif moola <= 103450.50 and moola >= 86208.75:
            return int(6)
        elif moola <= 120692.25 and moola >= 103450.51:
            return int(7)
        else:
            return 'wtf'    
    except:
        pass

df['INCOME_TRANSFORMED'] = df['INCOME_TRANSFORMED'].apply(yay)

x = df[['MAIN', 'HIGHDEG','CONTROL', 'AGE_GROUPED', 'MARRAIGE_LUMPED', 'INCOME_TRANSFORMED', 'NUMBRANCH']]
y = df['percent_on_student_loan']

rfr = RandomForestRegressor()

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.4)

rfr.fit(x_train, y_train)

rfr.score(x_train, y_train)

rfr.score(x_test, y_test)

parameters = [{'n_estimators': [10, 25, 50, 100, 500], 'max_depth': [2, 3, 5, 7, 9],
                     'max_features': [0.25, 0.5, 0.75, 1.0]}]

gridsearch = GridSearchCV(rfr, param_grid=parameters, cv=7, n_jobs=-1)

gridsearch.fit(x_train,y_train)

gridsearch.best_estimator_

gridsearch.best_estimator_.fit(x_train, y_train) 

gridsearch.best_estimator_.score(x_train, y_train)

gridsearch.best_estimator_.score(x_test, y_test)

df_test_set = pd.read_csv('university_test.csv')

# def evaluation_transformation(df, df_test_set):
#     dataset = df.join(pd.DataFrame(df_test_set, columns=['Prediction']))
#     dataset[['Id', 'Prediction']].to_csv('Project_5_Regression.csv', index=False)

# print dataset

# will continue to work on this to export predictions as a csv



